In [71]:

import pandas as pd
sider_df = pd.read_csv('meddra_all_se.tsv', sep='\t', header=None, names=['stitch_id', 'umls_id', 'side_effect'])
print(sider_df.head())

                                   stitch_id   umls_id            side_effect
CID100000085 CID000010917 C0000729       LLT  C0000729       Abdominal cramps
                          C0000729        PT  C0000737         Abdominal pain
                          C0000737       LLT  C0000737         Abdominal pain
                          C0000737        PT  C0687713  Gastrointestinal pain
                          C0000737        PT  C0000737         Abdominal pain


In [72]:
import time
import random
from pubchempy import get_compounds

def fetch_pubchem_data(drug_name, retries=5, delay=5):
    for attempt in range(retries):
        try:
            print(f"🔍 Fetching PubChem data for {drug_name} (Attempt {attempt+1}/{retries})...")
            compounds = get_compounds(drug_name, 'name')
            
            if not compounds:
                print(f"⚠️ No PubChem data found for {drug_name}. Trying lowercase version...")
                compounds = get_compounds(drug_name.lower(), 'name')
            
            if not compounds:
                print(f"❌ Still no data for {drug_name}. Trying synonyms...")
                synonyms = {
                    "gamma-aminobutyric": "GABA",
                    "5-aminolevulinic": "5-ALA",
                    "Oestrogen": "Estrogen"
                }
                if drug_name in synonyms:
                    compounds = get_compounds(synonyms[drug_name], 'name')
                    if compounds:
                        print(f"✅ Found {synonyms[drug_name]} instead of {drug_name}")
                        return compounds[0].isomeric_smiles
                    else:
                        print(f"❌ No data for synonym {synonyms[drug_name]}")

                return None  # No valid PubChem data

            return compounds[0].isomeric_smiles  # Return SMILES if found

        except Exception as e:
            if "PUGREST.ServerBusy" in str(e) or "Remote end closed" in str(e):
                print(f"⚠️ Server busy, retrying {drug_name} in {delay} seconds... (Attempt {attempt+1}/{retries})")
                time.sleep(delay + random.uniform(0, 3))  # Add random delay to avoid API blocks
            else:
                print(f"⚠️ Error fetching {drug_name}: {e}")
                return None  # Other errors won't retry

    print(f"❌ Failed after {retries} attempts: {drug_name}")
    return None  # Failed after all retries

In [73]:
print(fetch_pubchem_data("Aspirin"))

🔍 Fetching PubChem data for Aspirin (Attempt 1/5)...
CC(=O)OC1=CC=CC=C1C(=O)O


In [74]:
import pandas as pd

# Load drug names data
drug_names = pd.read_csv('drug_names.tsv', sep='\t', header=None, names=['stitch_id', 'drug_name'])

# Verify the data
print(drug_names.head())
merged_df = sider_df.merge(drug_names, on='stitch_id', how='inner')

      stitch_id                 drug_name
0  CID100000085                 carnitine
1  CID100000119        gamma-aminobutyric
2  CID100000137          5-aminolevulinic
3  CID100000143                leucovorin
4  CID100000146  5-methyltetrahydrofolate


In [75]:
merged_df = sider_df.merge(drug_names, on='stitch_id', how='inner')
merged_df

,stitch_id,umls_id,side_effect,drug_name


In [76]:
from rdkit import Chem
from rdkit.Chem import rdmolops
import torch
from torch_geometric.data import Data

def generate_molecular_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    adj = rdmolops.GetAdjacencyMatrix(mol)
    edge_index = torch.tensor(adj.nonzero(), dtype=torch.long)
    x = torch.tensor([atom.GetAtomicNum() for atom in mol.GetAtoms()], dtype=torch.float).view(-1, 1)
    return Data(x=x, edge_index=edge_index)

In [77]:
print(generate_molecular_graph(fetch_pubchem_data("Aspirin")))

🔍 Fetching PubChem data for Aspirin (Attempt 1/5)...
Data(x=[13, 1], edge_index=[2, 26])


In [78]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class DrugGNN(torch.nn.Module):
    def __init__(self):
        super(DrugGNN, self).__init__()
        self.conv1 = GCNConv(1, 64)
        self.conv2 = GCNConv(64, 32)
        self.fc = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return torch.sigmoid(self.fc(x))

In [ ]:
def preprocess_data(sider_df, drug_names_path):
 
    drug_names = pd.read_csv(drug_names_path, sep='\t', header=None, names=['stitch_id', 'drug_name'])
    data_list = []

    for _, row in drug_names.iterrows():
        try:
            smiles = fetch_pubchem_data(row['drug_name'])
            if smiles is None:
                continue
            side_effects = sider_df[sider_df['stitch_id'] == row['stitch_id']]['side_effect'].values
            graph = generate_molecular_graph(smiles)
            graph.y = torch.tensor([len(side_effects)], dtype=torch.float)
            data_list.append(graph)
        except Exception as e:
            print(f"⚠️ Error processing {row['drug_name']}: {e}")

    print(f"✅ Preprocessing Completed: {len(data_list)} graphs created.")
    return data_list
import pandas as pd
import torch

# Load SIDER Data First
sider_df = pd.read_csv('meddra_all_se.tsv', sep='\t', header=None, names=['stitch_id', 'umls_id', 'side_effect'])

# Now Call preprocess_data with sider_df
data_list = preprocess_data(sider_df, 'drug_names.tsv')

print(f"✅ Final Data List Count: {len(data_list)}")

In [28]:
from pubchempy import get_compounds

def fetch_pubchem_data(drug_name):
    try:
        # Try fetching data normally
        compounds = get_compounds(drug_name, 'name')
        if not compounds:
            print(f"⚠️ No PubChem data found for {drug_name}. Trying lowercase version...")
            
            # Try lowercase version
            compounds = get_compounds(drug_name.lower(), 'name')
            if not compounds:
                print(f"❌ Still no data for {drug_name}. Trying synonyms...")

                # If synonyms exist, try alternative search
                synonyms = {
                    "gamma-aminobutyric": "GABA",
                    "5-aminolevulinic": "5-ALA",
                    "Oestrogen": "Estrogen"
                }
                
                if drug_name in synonyms:
                    compounds = get_compounds(synonyms[drug_name], 'name')
                    if compounds:
                        print(f"✅ Found {synonyms[drug_name]} instead of {drug_name}")
                        return compounds[0].isomeric_smiles
                    else:
                        print(f"❌ No data for synonym {synonyms[drug_name]}")

                return None  # If all attempts fail
        return compounds[0].isomeric_smiles  # Return valid SMILES if found
    except Exception as e:
        print(f"⚠️ Error fetching {drug_name}: {e}")
        return None

In [29]:
missing_drugs = []
for _, row in drug_names.iterrows():
    smiles = fetch_pubchem_data(row['drug_name'])
    if smiles is None:
        missing_drugs.append(row['drug_name'])

print(f"⚠️ Missing PubChem Data for {len(missing_drugs)} drugs: {missing_drugs[:10]}")  # Show first 10

⚠️ No PubChem data found for gamma-aminobutyric. Trying lowercase version...
❌ Still no data for gamma-aminobutyric. Trying synonyms...
✅ Found GABA instead of gamma-aminobutyric
⚠️ No PubChem data found for 5-aminolevulinic. Trying lowercase version...
❌ Still no data for 5-aminolevulinic. Trying synonyms...
✅ Found 5-ALA instead of 5-aminolevulinic
⚠️ Error fetching betaine: 'PUGREST.ServerBusy'
⚠️ No PubChem data found for Oestrogen. Trying lowercase version...
❌ Still no data for Oestrogen. Trying synonyms...
✅ Found Estrogen instead of Oestrogen
⚠️ No PubChem data found for nitrous. Trying lowercase version...
❌ Still no data for nitrous. Trying synonyms...
⚠️ No PubChem data found for hydroxyl. Trying lowercase version...
❌ Still no data for hydroxyl. Trying synonyms...
⚠️ No PubChem data found for acetohydroxamic. Trying lowercase version...
❌ Still no data for acetohydroxamic. Trying synonyms...
⚠️ No PubChem data found for actinomycin. Trying lowercase version...
❌ Still no da

KeyboardInterrupt: 

In [23]:
print(sider_df.head())  
print(drug_names.head())

                                   stitch_id   umls_id            side_effect
CID100000085 CID000010917 C0000729       LLT  C0000729       Abdominal cramps
                          C0000729        PT  C0000737         Abdominal pain
                          C0000737       LLT  C0000737         Abdominal pain
                          C0000737        PT  C0687713  Gastrointestinal pain
                          C0000737        PT  C0000737         Abdominal pain
      stitch_id                 drug_name
0  CID100000085                 carnitine
1  CID100000119        gamma-aminobutyric
2  CID100000137          5-aminolevulinic
3  CID100000143                leucovorin
4  CID100000146  5-methyltetrahydrofolate


In [17]:
print(f"Number of graphs in data_list: {len(data_list)}")
if len(data_list) == 0:
    print("❌ No data found! Check preprocessing step.")

NameError: name 'data_list' is not defined

In [7]:
import pandas as pd

drug_names = pd.read_csv('drug_names.tsv', sep='\\t', header=None, names=['stitch_id', 'drug_name'])
# print(drug_names.head())  # Display the first few rows
drug_names.columns = drug_names.columns.str.strip()

from pubchempy import get_compounds

compound = get_compounds('Aspirin', 'name')[0]
print(compound.to_dict())

Index(['stitch_id', 'drug_name'], dtype='object')
Index(['stitch_id', 'drug_name'], dtype='object')
      stitch_id                 drug_name
0  CID100000085                 carnitine
1  CID100000119        gamma-aminobutyric
2  CID100000137          5-aminolevulinic
3  CID100000143                leucovorin
4  CID100000146  5-methyltetrahydrofolate


/var/folders/z2/wl288m9n139_4ywpb2k5s8t40000gn/T/ipykernel_17507/144763402.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  drug_names = pd.read_csv('drug_names.tsv', sep='\\t', header=None, names=['stitch_id', 'drug_name'])


In [57]:
missing_drugs = []

def preprocess_data(sider_df, drug_names_path):
    drug_names = pd.read_csv(drug_names_path, sep='\t', header=None, names=['stitch_id', 'drug_name'])
    data_list = []

    for _, row in drug_names.iterrows():
        try:
            smiles = fetch_pubchem_data(row['drug_name'])
            if smiles is None:
                missing_drugs.append(row['drug_name'])
                continue
            side_effects = sider_df[sider_df['stitch_id'] == row['stitch_id']]['side_effect'].values
            graph = generate_molecular_graph(smiles)
            graph.y = torch.tensor([len(side_effects)], dtype=torch.float)
            data_list.append(graph)
        except Exception as e:
            print(f"⚠️ Error processing {row['drug_name']}: {e}")

    print(f"✅ Preprocessing Completed: {len(data_list)} graphs created.")
    print(f"⚠️ {len(missing_drugs)} drugs missing data: {missing_drugs[:10]} (Showing first 10)")
    return data_list

In [51]:
drug_names = pd.read_csv('drug_names.tsv', sep='\t', header=None, names=['stitch_id', 'drug_name'])
print("✅ Drug Names Loaded:")
print(drug_names.head())  # Ensure it has correct data

✅ Drug Names Loaded:
      stitch_id                 drug_name
0  CID100000085                 carnitine
1  CID100000119        gamma-aminobutyric
2  CID100000137          5-aminolevulinic
3  CID100000143                leucovorin
4  CID100000146  5-methyltetrahydrofolate


In [52]:
for _, row in drug_names.iterrows():
    print(f"🔍 Fetching data for: {row['drug_name']}")
    smiles = fetch_pubchem_data(row['drug_name'])

    if smiles is None:
        print(f"❌ No SMILES found for {row['drug_name']}")
        continue
    print(f"✅ SMILES found: {smiles}")

🔍 Fetching data for: carnitine
🔍 Fetching PubChem data for carnitine (Attempt 1/5)...
✅ SMILES found: C[N+](C)(C)CC(CC(=O)[O-])O
🔍 Fetching data for: gamma-aminobutyric
🔍 Fetching PubChem data for gamma-aminobutyric (Attempt 1/5)...
⚠️ No PubChem data found for gamma-aminobutyric. Trying lowercase version...
❌ Still no data for gamma-aminobutyric. Trying synonyms...
✅ Found GABA instead of gamma-aminobutyric
✅ SMILES found: C(CC(=O)O)CN
🔍 Fetching data for: 5-aminolevulinic
🔍 Fetching PubChem data for 5-aminolevulinic (Attempt 1/5)...
⚠️ No PubChem data found for 5-aminolevulinic. Trying lowercase version...
❌ Still no data for 5-aminolevulinic. Trying synonyms...
✅ Found 5-ALA instead of 5-aminolevulinic
✅ SMILES found: C(CC(=O)O)C(=O)CN
🔍 Fetching data for: leucovorin
🔍 Fetching PubChem data for leucovorin (Attempt 1/5)...
✅ SMILES found: C1C(N(C2=C(N1)N=C(NC2=O)N)C=O)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O
🔍 Fetching data for: 5-methyltetrahydrofolate
🔍 Fetching PubChem data for

KeyboardInterrupt: 

In [54]:
test_smiles = fetch_pubchem_data("Aspirin")
print(f"✅ Aspirin SMILES: {test_smiles}")

🔍 Fetching PubChem data for Aspirin (Attempt 1/5)...
✅ Aspirin SMILES: CC(=O)OC1=CC=CC=C1C(=O)O


In [62]:
# Ensure SIDER data is loaded first
sider_df = pd.read_csv('meddra_all_se.tsv', sep='\t', header=None, names=['stitch_id', 'umls_id', 'side_effect'])

# Process data and create molecular graphs
data_list = preprocess_data(sider_df, 'drug_names.tsv')

# Check if data_list is populated
if 'data_list' in locals():
    print(data_list[:5])
    # print(f"✅ Total Processed Drugs: {len(data_list)}")
    print("🔍 Sample Data:")
    # print(data_list[:5])  # Show first few processed molecular graphs
else:
    print("❌ `data_list` is not defined. Check if `preprocess_data()` is running correctly.")

🔍 Fetching PubChem data for carnitine (Attempt 1/5)...
🔍 Fetching PubChem data for gamma-aminobutyric (Attempt 1/5)...
⚠️ No PubChem data found for gamma-aminobutyric. Trying lowercase version...
❌ Still no data for gamma-aminobutyric. Trying synonyms...
✅ Found GABA instead of gamma-aminobutyric
🔍 Fetching PubChem data for 5-aminolevulinic (Attempt 1/5)...
⚠️ No PubChem data found for 5-aminolevulinic. Trying lowercase version...
❌ Still no data for 5-aminolevulinic. Trying synonyms...
✅ Found 5-ALA instead of 5-aminolevulinic
🔍 Fetching PubChem data for leucovorin (Attempt 1/5)...
🔍 Fetching PubChem data for 5-methyltetrahydrofolate (Attempt 1/5)...
🔍 Fetching PubChem data for PGE2 (Attempt 1/5)...
🔍 Fetching PubChem data for prostacyclin (Attempt 1/5)...
🔍 Fetching PubChem data for prostaglandin (Attempt 1/5)...
🔍 Fetching PubChem data for acetate (Attempt 1/5)...
🔍 Fetching PubChem data for acetylcholine (Attempt 1/5)...
🔍 Fetching PubChem data for adenosine (Attempt 1/5)...
🔍 Fetc

[22:07:16] WARNING: not removing hydrogen atom without neighbors
[22:07:16] WARNING: not removing hydrogen atom without neighbors


🔍 Fetching PubChem data for ambrisentan (Attempt 1/5)...
🔍 Fetching PubChem data for L-Dmp (Attempt 1/5)...
⚠️ No PubChem data found for L-Dmp. Trying lowercase version...
❌ Still no data for L-Dmp. Trying synonyms...
🔍 Fetching PubChem data for dronedarone (Attempt 1/5)...
🔍 Fetching PubChem data for ramelteon (Attempt 1/5)...
🔍 Fetching PubChem data for lapatinib (Attempt 1/5)...
🔍 Fetching PubChem data for NuvaRing (Attempt 1/5)...
🔍 Fetching PubChem data for dabigatran (Attempt 1/5)...
🔍 Fetching PubChem data for darunavir (Attempt 1/5)...
🔍 Fetching PubChem data for lurasidone (Attempt 1/5)...
🔍 Fetching PubChem data for luliconazole (Attempt 1/5)...
🔍 Fetching PubChem data for aliskiren (Attempt 1/5)...
🔍 Fetching PubChem data for dabigatran (Attempt 1/5)...
🔍 Fetching PubChem data for sitaxsentan (Attempt 1/5)...
🔍 Fetching PubChem data for tolvaptan (Attempt 1/5)...
🔍 Fetching PubChem data for sorafenib (Attempt 1/5)...
🔍 Fetching PubChem data for Colimycin (Attempt 1/5)...
🔍 F

[22:07:22] WARNING: not removing hydrogen atom without neighbors
[22:07:22] WARNING: not removing hydrogen atom without neighbors


🔍 Fetching PubChem data for fosaprepitant (Attempt 1/5)...
🔍 Fetching PubChem data for cortisone (Attempt 1/5)...
🔍 Fetching PubChem data for 18F-FDG (Attempt 1/5)...
⚠️ No PubChem data found for 18F-FDG. Trying lowercase version...
❌ Still no data for 18F-FDG. Trying synonyms...
🔍 Fetching PubChem data for trimethoprim-sulfamethoxazole (Attempt 1/5)...
🔍 Fetching PubChem data for carboprost (Attempt 1/5)...
🔍 Fetching PubChem data for natamycin (Attempt 1/5)...
🔍 Fetching PubChem data for CAS (Attempt 1/5)...
⚠️ No PubChem data found for CAS. Trying lowercase version...
❌ Still no data for CAS. Trying synonyms...
🔍 Fetching PubChem data for cefditoren (Attempt 1/5)...
🔍 Fetching PubChem data for gadoversetamide (Attempt 1/5)...
🔍 Fetching PubChem data for ciclesonide (Attempt 1/5)...
🔍 Fetching PubChem data for roflumilast (Attempt 1/5)...
🔍 Fetching PubChem data for hemin (Attempt 1/5)...
🔍 Fetching PubChem data for FK463 (Attempt 1/5)...
🔍 Fetching PubChem data for maraviroc (Attemp

[22:08:11] WARNING: not removing hydrogen atom without neighbors
[22:08:11] WARNING: not removing hydrogen atom without neighbors
[22:08:11] WARNING: not removing hydrogen atom without neighbors


🔍 Fetching PubChem data for trametinib (Attempt 1/5)...
🔍 Fetching PubChem data for spinosad (Attempt 1/5)...
⚠️ No PubChem data found for spinosad. Trying lowercase version...
❌ Still no data for spinosad. Trying synonyms...
🔍 Fetching PubChem data for nitroprusside (Attempt 1/5)...
🔍 Fetching PubChem data for empagliflozin (Attempt 1/5)...
🔍 Fetching PubChem data for vasopressin (Attempt 1/5)...
🔍 Fetching PubChem data for ulipristal (Attempt 1/5)...
🔍 Fetching PubChem data for MDV3100 (Attempt 1/5)...
🔍 Fetching PubChem data for macitentan (Attempt 1/5)...
🔍 Fetching PubChem data for TMC435 (Attempt 1/5)...
🔍 Fetching PubChem data for PCI-32765 (Attempt 1/5)...
🔍 Fetching PubChem data for salmon (Attempt 1/5)...
⚠️ No PubChem data found for salmon. Trying lowercase version...
❌ Still no data for salmon. Trying synonyms...
🔍 Fetching PubChem data for cosyntropin (Attempt 1/5)...
🔍 Fetching PubChem data for LY146032 (Attempt 1/5)...
🔍 Fetching PubChem data for Nuvocid (Attempt 1/5)...

In [65]:
print(f"✅ Total Processed Drugs: {len(data_list)}")
print("🔍 Sample Data:")
for data in data_list[:5]:  # Show first 5 molecular graphs
    print(f"📌 Graph - x: {data.x.shape}, edge_index: {data.edge_index.shape}, y: {data.y}")

✅ Total Processed Drugs: 1357
🔍 Sample Data:
📌 Graph - x: torch.Size([11, 1]), edge_index: torch.Size([2, 20]), y: tensor([0.])
📌 Graph - x: torch.Size([7, 1]), edge_index: torch.Size([2, 12]), y: tensor([0.])
📌 Graph - x: torch.Size([9, 1]), edge_index: torch.Size([2, 16]), y: tensor([0.])
📌 Graph - x: torch.Size([34, 1]), edge_index: torch.Size([2, 72]), y: tensor([0.])
📌 Graph - x: torch.Size([33, 1]), edge_index: torch.Size([2, 70]), y: tensor([0.])


In [66]:
test_smiles = "CC(C(=O)O)N"  # Example: Alanine
test_graph = generate_molecular_graph(test_smiles)

print(f"✅ Test Graph - x: {test_graph.x.shape}, edge_index: {test_graph.edge_index.shape}")

✅ Test Graph - x: torch.Size([6, 1]), edge_index: torch.Size([2, 10])


In [68]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

# Define Graph Neural Network Model
class DrugGNN(torch.nn.Module):
    def __init__(self):
        super(DrugGNN, self).__init__()
        self.conv1 = GCNConv(1, 64)  # Input: Atomic number
        self.conv2 = GCNConv(64, 32)
        self.fc = torch.nn.Linear(32, 1)  # Predict side effect severity

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return torch.sigmoid(self.fc(x))  # Binary output (Side effect likelihood)

# ✅ Move `train_model()` Outside the Class
def train_model(data_list, epochs=20, batch_size=32, learning_rate=0.001):
    print("🚀 Starting Training...")

    # Initialize model, optimizer, and loss function
    model = DrugGNN()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)

            # Fix shape mismatch
            out = out.mean(dim=0, keepdim=True)
            y = batch.y.view(-1, 1).mean(dim=0, keepdim=True)

            loss = F.mse_loss(out, y)  # Mean Squared Error Loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"✅ Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(loader)}")

    print("🎉 Training Complete!")

# Run Training
train_model(data_list)

🚀 Starting Training...
✅ Epoch 1/20, Loss: 0.10723078558438046
✅ Epoch 2/20, Loss: 0.008417007367148302
✅ Epoch 3/20, Loss: 0.002033850524661153
✅ Epoch 4/20, Loss: 0.00099642998755498
✅ Epoch 5/20, Loss: 0.0006038166425559063
✅ Epoch 6/20, Loss: 0.0004074883425261739
✅ Epoch 7/20, Loss: 0.00029422873820633043
✅ Epoch 8/20, Loss: 0.00022350502509060641
✅ Epoch 9/20, Loss: 0.00017497404431112025
✅ Epoch 10/20, Loss: 0.00014120884876573717
✅ Epoch 11/20, Loss: 0.00011655304809019706
✅ Epoch 12/20, Loss: 9.830793275140486e-05
✅ Epoch 13/20, Loss: 8.35035006762049e-05
✅ Epoch 14/20, Loss: 7.196269566626396e-05
✅ Epoch 15/20, Loss: 6.226800296856307e-05
✅ Epoch 16/20, Loss: 5.485645661517696e-05
✅ Epoch 17/20, Loss: 4.8717912905822444e-05
✅ Epoch 18/20, Loss: 4.3196469304411736e-05
✅ Epoch 19/20, Loss: 3.871071072184307e-05
✅ Epoch 20/20, Loss: 3.4935800042233515e-05
🎉 Training Complete!


In [69]:
def evaluate_model(model, test_data):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No gradients needed during evaluation
        for data in test_data:
            out = model(data.x, data.edge_index)
            out = out.mean(dim=0, keepdim=True)  # Ensure output matches y
            y = data.y.view(-1, 1).mean(dim=0, keepdim=True)
            loss = F.mse_loss(out, y)
            total_loss += loss.item()

    avg_loss = total_loss / len(test_data)
    print(f"🔍 Model Evaluation Loss: {avg_loss}")

# Prepare test dataset (use a subset of data_list as a quick check)
test_data = data_list[:10]  # Use 10 samples for testing
evaluate_model(DrugGNN(), test_data)

🔍 Model Evaluation Loss: 0.2675763934850693


In [70]:
def predict_side_effects(drug_name, model):
    smiles = fetch_pubchem_data(drug_name)
    if smiles is None:
        print(f"❌ No PubChem data found for {drug_name}")
        return None

    graph = generate_molecular_graph(smiles)
    model.eval()
    with torch.no_grad():
        prediction = model(graph.x, graph.edge_index).mean().item()
    
    print(f"🔬 Predicted Side Effect Severity for {drug_name}: {prediction:.4f}")
    return prediction

# Try predicting for Aspirin
trained_model = DrugGNN()
predict_side_effects("Aspirin", trained_model)

🔍 Fetching PubChem data for Aspirin (Attempt 1/5)...
🔬 Predicted Side Effect Severity for Aspirin: 0.5417


0.5416805744171143